## Part 2 


#### Dictionary containing the occurrences of each annotation in the training set

In [1]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}
with open("train", encoding="utf-8") as file:
    my_list = file.readlines()
    my_list = [x.strip() for x in my_list]
    for i in range(len(my_list)):
        annotated_word = my_list[i]
        each_word = annotated_word.split(" ")
        if (len(each_word) == 2):
            if each_word[1] == "O":
                annotationDict["O"] += 1
            elif each_word[1] == "I-positive":
                annotationDict["I-positive"] += 1
            elif each_word[1] == "B-positive":
                annotationDict["B-positive"] += 1
            elif each_word[1] == "I-negative":
                annotationDict["I-negative"] += 1
            elif each_word[1] == "B-negative":
                annotationDict["B-negative"] += 1
            elif each_word[1] == "I-neutral":
                annotationDict["I-neutral"] += 1
            elif each_word[1] == "B-neutral":
                annotationDict["B-neutral"] += 1

    print(annotationDict)

{'O': 24242, 'I-positive': 607, 'B-positive': 1208, 'I-negative': 133, 'B-negative': 382, 'I-neutral': 23, 'B-neutral': 65}


#### (a) Write a function that estimates the emission parameters from the training set using MLE

In [2]:
def estimate_emission(word, tag):
    with open("train", encoding="utf-8") as file:
        my_list = file.readlines()
        my_list = [x.strip() for x in my_list]
        countTop = 0
        countBottom = annotationDict[tag]
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            if len(each_word) == 2:
                if (each_word[0] == word and each_word[1] == tag):
                    countTop += 1
                    
#         print(countTop)
#         print(countBottom)
        return countTop/countBottom

estimate_emission("service", "O")

0.0004950086626515964

#### (b) Replace the words that appear less than k times in the training set with a special token #UNK# before training. This leads to a “modified training set”.

In [3]:
import datetime

def modify_trainingset(k):
    with open("train", encoding="utf-8") as file:
        my_list = file.readlines()
        my_list = [x.strip() for x in my_list]
        wordCountDict = {}
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            if each_word[0] in wordCountDict:
                wordCountDict[each_word[0]] += 1
            else:
                wordCountDict[each_word[0]] = 1
        
       
        wordToBeReplacedDict = {}
        for key in wordCountDict:
            if wordCountDict[key] < k:
                wordToBeReplacedDict[key] = wordCountDict[key]

        start = datetime.datetime.now()
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            for key in wordToBeReplacedDict:
                if each_word[0] == key:
                    my_list[i] = "#UNK# " + each_word[1]

        modifiedTotalTrainSet = "\n".join(my_list)

        f = open("modified_train_final2","w+")
        f.write(modifiedTotalTrainSet)
                            
        end = datetime.datetime.now()
        elapsed = end - start
        print(elapsed)
            

    
modify_trainingset(3)


0:00:04.732046


In [4]:
def load_modified_train_file():
    with open("modified_train_final2", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in my_list]
        return modified_train_list
        
modified_train_list = load_modified_train_file()

In [5]:
def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modified_train_list)):
        annotated_word = modified_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    
modifiedWordDict = get_word_dict()

In [6]:
def estimate_emission_fix(word, tag):
        countTop = 0
        countBottom = annotationDict[tag]
        if word in modifiedWordDict:
            # do nothing
            word = word
            
        else:
            word = "#UNK#"
            
        for i in range(len(modified_train_list)):
            annotated_word = modified_train_list[i]
            each_word = annotated_word.split(" ")
            if (each_word[0] == word and each_word[1] == tag):
                countTop += 1
#         print('emission')
#         print(countTop)
        return countTop/countBottom

estimate_emission_fix("When", "B-negative")

0.0

#### (c) Implement a simple sentiment analysis system that produces the tag

In [7]:
def sentiment_analysis(word):
    OScore = estimate_emission_fix(word, "O")
    BPlusScore = estimate_emission_fix(word, "B-positive")
    IPlusScore = estimate_emission_fix(word, "I-positive")
    BMinusScore = estimate_emission_fix(word, "B-negative")
    IMinusScore = estimate_emission_fix(word, "I-negative")
    BNeutralScore = estimate_emission_fix(word, "B-neutral")
    INeutralScore = estimate_emission_fix(word, "I-neutral")

    scoreDict = {"O": OScore, "I-positive": IPlusScore, "B-positive": BPlusScore, "I-negative":IMinusScore, "B-negative": BMinusScore, "I-neutral":INeutralScore, "B-neutral":BNeutralScore}
    maximumScore = max(scoreDict.values())
    return max(scoreDict, key=scoreDict.get)

   
sentiment_analysis("service")

'B-negative'

In [8]:
def load_files():
    with open("dev.in", encoding="utf-8") as file:
        dev_in_list = file.readlines()
        dev_in_list = [x.strip() for x in my_list]
        return dev_in_list
        
dev_in_list = load_files()

In [9]:
import datetime
def predict_sentiment():
    wordDict = {}
    f = open("devfinal.pfinal.out","w+")
    start = datetime.datetime.now()
    for i in range(len(dev_in_list)):
        each_word = dev_in_list[i]
        if (i > 4300):
            print(". ",end="")
        if each_word in wordDict:
            tag = wordDict[each_word]
            f.write(each_word +  " " + tag + "\n")
        else:
            if each_word != "":
                tag = sentiment_analysis(each_word)
                wordDict[each_word] = tag 
                f.write(each_word +  " " + tag + "\n")
            else:
                f.write(" \n")

    end = datetime.datetime.now()
    elapsed = end - start
    print(elapsed)

predict_sentiment()

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

#### Calculate F score, use evalScript

### Part 3 
#### Function to estimate Transition Parameters using MLE

In [146]:
def separate_tweets():
    with open("modified_train_final2", encoding="utf-8") as file:
        my_list = file.readlines()
        my_list = [x.strip() for x in my_list]
#         because last sentence has no ''
        countStart = 1
        countBottom = 0
        countTop = 0
        # counting starts
        giant_each_tweet = []
        each_tweet = []
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            if each_word == ['']:
                countStart += 1
                giant_each_tweet.append(each_tweet)
                each_tweet = []
                
            else:
                each_tweet.append(each_word[1])
        for i in range(len(giant_each_tweet)):
            giant_each_tweet[i].insert(0, 'START')
            giant_each_tweet[i].append('STOP')
        return giant_each_tweet, countStart
giant_each_tweet, countStart = separate_tweets()

In [179]:
def estimate_transition(current_tag, previous_tag):
#     with open("modified_train_final2", encoding="utf-8") as file:
    startDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}
    for i in range(len(giant_each_tweet)):
        each_tweet = giant_each_tweet[i]
        if each_tweet[1] == "O":
            startDict["O"] += 1
        elif each_tweet[1] == "I-positive":
            startDict["I-positive"] += 1
        elif each_tweet[1] == "B-positive":
            startDict["B-positive"] += 1
        elif each_tweet[1] == "I-negative":
            startDict["I-negative"] += 1
        elif each_tweet[1] == "B-negative":
            startDict["B-negative"] += 1
        elif each_tweet[1] == "I-neutral":
            startDict["I-neutral"] += 1
        elif each_tweet[1] == "B-neutral":
            startDict["B-neutral"] += 1

    stopDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}
    for i in range(len(giant_each_tweet)):
        each_tweet = giant_each_tweet[i]
        last_index = len(each_tweet) - 2
        if each_tweet[last_index] == "O":
            stopDict["O"] += 1
        elif each_tweet[last_index] == "I-positive":
            stopDict["I-positive"] += 1
        elif each_tweet[last_index] == "B-positive":
            stopDict["B-positive"] += 1
        elif each_tweet[last_index] == "I-negative":
            stopDict["I-negative"] += 1
        elif each_tweet[last_index] == "B-negative":
            stopDict["B-negative"] += 1
        elif each_tweet[last_index] == "I-neutral":
            stopDict["I-neutral"] += 1
        elif each_tweet[last_index] == "B-neutral":
            stopDict["B-neutral"] += 1

    if previous_tag == "START":
        # current tag is Yn
        countTop = startDict[current_tag]
#             print(countTop)
#             print(countStart)
#             print(countTop/countStart)
        return countTop/countStart

    elif current_tag == "STOP":
        # previous tag is Yn
        countTop = stopDict[previous_tag]
        countBottom = annotationDict[previous_tag]
        return countTop/countBottom

    else:
        giantTransitionDict = {}
        for i in range(len(giant_each_tweet)):
            each_tweet = giant_each_tweet[i]
            for j in range(1,len(each_tweet)-1):
                if j == len(each_tweet):
                    break
                if each_tweet[j-1] == previous_tag and each_tweet[j] == "O":
                    if (previous_tag + "O") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "O"] += 1
                    else:
                        giantTransitionDict[previous_tag + "O"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] == "B-negative":
                    if (previous_tag + "B-negative") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "B-negative"] += 1
                    else:
                        giantTransitionDict[previous_tag + "B-negative"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] == "B-positive":
                    if (previous_tag + "B-positive") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "B-positive"] += 1
                    else:
                        giantTransitionDict[previous_tag + "B-positive"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] =="B-neutral":
                    if (previous_tag + "B-neutral") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "B-neutral"] += 1
                    else:
                        giantTransitionDict[previous_tag + "B-neutral"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] =="I-positive":
                    if (previous_tag + "I-positive") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "I-positive"] += 1
                    else:
                        giantTransitionDict[previous_tag + "I-positive"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] =="I-negative":
                    if (previous_tag + "I-negative") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "I-negative"] += 1
                    else:
                        giantTransitionDict[previous_tag + "I-negative"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] =="I-neutral":
                    if (previous_tag + "I-neutral") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "I-neutral"] += 1
                    else:
                        giantTransitionDict[previous_tag + "I-neutral"] = 1
#             print(giantTransitionDict)
        if giantTransitionDict.get(previous_tag+current_tag) == None:
            return 0
        else:
            countTop = giantTransitionDict[previous_tag+current_tag]
            countBottom = annotationDict[previous_tag]

            return countTop/countBottom

    return 1
# estimate_transition("O", "START")
# ineg = estimate_transition("STOP", "I-negative")
# ipos = estimate_transition("STOP", "I-positive")
# bneg = estimate_transition("STOP", "B-negative")
# bpos = estimate_transition("STOP", "B-positive")
# ineu = estimate_transition("STOP", "I-neutral")
# bneu = estimate_transition("STOP", "B-neutral")
# o = estimate_transition("STOP", "O")
# result = ineg + ipos + bneg + bpos +ineu + bneu + o
# print(result)
estimate_transition("O", "START") #current, previous

0.8731322244149986

In [178]:
import operator
def viterbi():
    with open("dev.in", encoding="utf-8") as file:
        my_list = file.readlines()#read lines
        my_list = [x.strip() for x in my_list]
        alltags=["O","B-negative","B-positive","B-neutral","I-negative","I-positive","I-neutral"]
        thetags=[]
        piList = []
        n=["start"]
        beginning= True
        for i in range(len(my_list)):
            maxdict={}
            pi1=0
            pi2=0
            pi3=0
            pi4=0
            pi5=0
            pi6=0
            pi7=0#find some way to differentriate start
            if beginning: #base case
                k="START" #k is the starting tag
                maxdict["O"]=estimate_transition("O",k)*estimate_emission_fix(my_list[i],"O")
                maxdict["B-negative"]=estimate_transition("B-negative",k)*estimate_emission_fix(my_list[i],"B-negative")
                maxdict["B-positive"]=estimate_transition("B-positive",k)*estimate_emission_fix(my_list[i],"B-positive")
                maxdict["B-neutral"]=estimate_transition("B-neutral",k)*estimate_emission_fix(my_list[i],"B-neutral")
                maxdict["I-negative"]=estimate_transition("I-negative",k)*estimate_emission_fix(my_list[i],"I-negative")
                maxdict["I-positive"]=estimate_transition("I-positive",k)*estimate_emission_fix(my_list[i],"I-positive")
                maxdict["I-neutral"]=estimate_transition("I-neutral",k)*estimate_emission_fix(my_list[i],"I-neutral")
#                 highest=max(maxdict.iteritems(), key=operator.itemgetter(1))[0]
                piLayer = []
                #     dont need the dictionary
                pi1=maxdict["O"]
                piLayer.append(pi1)
                pi2=maxdict["B-negative"]
                piLayer.append(pi2)
                pi3=maxdict["B-positive"]
                piLayer.append(pi3)
                pi4=maxdict["B-neutral"]
                piLayer.append(pi4)
                pi5=maxdict["I-negative"]
                piLayer.append(pi5)
                pi6=maxdict["I-positive"]
                piLayer.append(pi6)
                pi7=maxdict["I-neutral"]
                piLayer.append(pi7)
                piList.append(piLayer)
                beginning = False
                print('PI LAYER:')
                print(piLayer)
                print(piList)
            else: #recursive cases
                piLayer = []
                for j in range(7):
                    nList = []
                    previous_tag = alltags[j]
                    n1 = piList[len(piList)-1][j]*estimate_transition("O", previous_tag)*estimate_emission_fix(my_list[i],previous_tag)
                    n2 = piList[len(piList)-1][j]*estimate_transition("B-negative", previous_tag)*estimate_emission_fix(my_list[i],previous_tag)
                    n3 = piList[len(piList)-1][j]*estimate_transition("B-positive", previous_tag)*estimate_emission_fix(my_list[i],previous_tag)
                    n4 = piList[len(piList)-1][j]*estimate_transition("B-neutral", previous_tag)*estimate_emission_fix(my_list[i],previous_tag)
                    n5 = piList[len(piList)-1][j]*estimate_transition("I-negative", previous_tag)*estimate_emission_fix(my_list[i],previous_tag)
                    n6 = piList[len(piList)-1][j]*estimate_transition("I-positive", previous_tag)*estimate_emission_fix(my_list[i],previous_tag)
                    n7 = piList[len(piList)-1][j]*estimate_transition("I-neutral", previous_tag)*estimate_emission_fix(my_list[i],previous_tag)
                    #find the max, put this into a dict/list
                    nList.append(n1)
                    nList.append(n2)
                    nList.append(n3)
                    nList.append(n4)
                    nList.append(n5)
                    nList.append(n6)
                    nList.append(n7)
                    maxN = max(nList)
                    print('MAX towards next node of tag ' + previous_tag)
                    print(maxN)
                    #append the max into piLayer2 and append piLayer2 into piList
                    piLayer.append(maxN)
                 
                print('LAYER HERE')
                print(piLayer)
                piList.append(piLayer)
            
            if my_list[i]=="":
                beginning= True
        print(piList)
              
                
viterbi()

PI LAYER:
[0.0004091930034968332, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[[0.0004091930034968332, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
MAX of 1N
3.811760554792914e-06
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
LAYER HERE
[3.811760554792914e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
LIST HERE
[[0.0004091930034968332, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.811760554792914e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
MAX of 1N
8.289744603926566e-10
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
LAYER HERE
[8.289744603926566e-10, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
LIST HERE
[[0.0004091930034968332, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.811760554792914e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [8.289744603926566e-10, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
MAX of 1N
3.5831404127665697e-12
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
MAX of 1N
0.0
LAYER HERE
[3.5831404127665697e-12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
LIST HERE
[[0.0004091930034968332, 0.0

KeyboardInterrupt: 